# Creating the Backend Graphs


In [4]:
from neo4j import GraphDatabase, basic_auth
import nxneo4j as nx #using nxneo4j
from py2neo import Graph, Node 
import pandas

### Creating the graphs

The Graphs are created using neo4j and hosted using db aura, where one can remote into. The following code connects to the db.

In [5]:
graph = GraphDatabase.driver("neo4j+s://101fd6b7.databases.neo4j.io", auth=basic_auth("neo4j","gB9F-fD2doYqInIcXR3DJZwnvvDWm-ZpgvOJ3BGCl54"))

cGraph = Graph("neo4j+s://101fd6b7.databases.neo4j.io", auth=('neo4j', "gB9F-fD2doYqInIcXR3DJZwnvvDWm-ZpgvOJ3BGCl54"))

In [6]:
G = nx.Graph(graph) #create the empty graph

In [7]:
fromFile = pandas.read_csv('../CSV/sbsRecipes3.csv')

### Preparing the data for nodes 

To start we need to get all the unique ingredients in the dataset. These will be used as ingredient nodes in the graph.

In [8]:
uniqueIng = []
for index, row in fromFile.iterrows():
    if row[2] not in uniqueIng:
        uniqueIng.append(row[2])

Next we find all the unique combinations of ingredients. These will be used for the edges later on

In [9]:
ingCombo = []
for curIndex in range(0, len(uniqueIng)):
    for tempIndex in range((curIndex + 1), len(uniqueIng)):
        tempCombo = []
        tempCombo.append(uniqueIng[curIndex])
        tempCombo.append(uniqueIng[tempIndex])
        ingCombo.append(tempCombo)

In [10]:
ingWeight = [0] * len(ingCombo)

Next we create a dictionary that contains all of the recipes as the keys, and all of the ingredients found in that recipe as the content. This will be used to connect the ingredients to there respective recipe in the graph.

In [11]:
curRecipe = ''
recIng = {}
ingList = []

for index, row in fromFile.iterrows():
    if curRecipe == row[0][0:-34]: # the data has a lot of black space after it. This removes it
        ingList.append(row[2])
    else:
        recIng[curRecipe] = ingList
        curRecipe = row[0][0:-34]
        ingList = []

recIng[curRecipe] = ingList
curRecipe = row[0][0:-34]
ingList = []
del recIng['']

The weight of the connection is calculated based on the amount of recipes the pair has in common. The next snippet of code does exactly that. The weight is then normalized to be between 0 and 1.

In [12]:
for ingComboIndex in range(0, len(ingCombo)):
    firstIng = ingCombo[ingComboIndex][0]
    secondIng = ingCombo[ingComboIndex][1]
    for recipe, ings in recIng.items():
        if firstIng in ings and secondIng in ings:
            ingWeight[ingComboIndex] += 1
        else:
            continue

In [13]:
maxWeight = max(ingWeight)

In [14]:
normWeights = [i/maxWeight for i in ingWeight]

In [15]:
firstIngs = []
secondIngs = []
for fI, sI in ingCombo:
    firstIngs.append(fI)
    secondIngs.append(sI)

The Data is saved to CSV so it can be accessed easily 

In [16]:
combosDataframe = pandas.DataFrame(list(zip(firstIngs, secondIngs, normWeights)),
               columns =['FirstIng', 'SecondIng', 'ComboWeight'])

In [17]:
combosDataframe.to_csv('../CSV/sbsCompComboWeights.csv', index=False)

In [18]:
compFirstIngs = []
compSecondIngs = []
compNormWeights = []
for normWeightsIndex in range(0, len(normWeights)):
    if normWeights[normWeightsIndex] != 0.0:
        compFirstIngs.append(firstIngs[normWeightsIndex])
        compSecondIngs.append(secondIngs[normWeightsIndex])
        compNormWeights.append(normWeights[normWeightsIndex])
    else:
        continue

In [19]:
compCombosDataframe = pandas.DataFrame(list(zip(compFirstIngs, compSecondIngs, compNormWeights)),
               columns =['FirstIng', 'SecondIng', 'ComboWeight'])
compCombosDataframe.to_csv('../CSV/sbsComboWeights.csv', index=False)

In [20]:
uniqueConIng = []
for index in range(0, len(compFirstIngs)):
    if compFirstIngs[index] not in uniqueConIng:
        uniqueConIng.append(compFirstIngs[index])

    if compSecondIngs[index] not in uniqueConIng:
        uniqueConIng.append(compSecondIngs[index])

In [21]:
print(len(uniqueConIng))
print(len(uniqueIng))

125
133


In [23]:
for currIng in uniqueIng:
    addNode = "CREATE (n:Ingredient {name : '" + currIng + "'})"

    cGraph.query(addNode)

In [24]:
for index in range(0,len(compFirstIngs)):
    G.add_edge(compFirstIngs[index], compSecondIngs[index], weight = compNormWeights[index])
    


In [25]:
query = "MATCH(n) RETURN COUNT(n)"

result = cGraph.query(query)
print(result)

 COUNT(n) 
----------
      399 

